<a href="https://colab.research.google.com/github/summer-0401/Coding_Test/blob/main/recommender_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. settings

https://greeksharifa.github.io/machine_learning/2020/06/01/LightFM/#21-data-preparation

In [1]:
# from try import random_train_test_split
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import pickle
import csv

import scipy.sparse as sp
from scipy.sparse import csr_matrix, coo_matrix
from scipy import *
try:
    from lightfm import LightFM
except:
    !pip install lightfm #이거 안 깔려 있으면 깔기
    from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score

import pickle
import re
import seaborn as sns

In [2]:
import gzip
import json
import re
import os
import sys

## 0.1. get path

In [3]:
from google.colab import drive
drive.mount('/content/drive')
basic_path = '/content/drive/MyDrive/recommender/'

DIR = basic_path + 'test3/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
# 1. process data

In [4]:
genre = 'children'
#genre = 'comics_graphic'
#genre = 'fantasy_paranormal' : 얘 안됨
#genre = 'history_biography'
#genre = 'mystery_thriller_crime'
#genre = 'poetry'
#genre = 'romance'
#genre = 'young_adult'

## 1.1. get file

In [5]:
books = pd.read_json(os.path.join(DIR, 'input/goodreads_books_%s.json.gz'%(genre)), lines = True, compression='gzip')

In [6]:
ratings = pd.read_json(os.path.join(DIR, 'input/goodreads_reviews_%s.json.gz'%(genre)), lines = True, compression='gzip', )

## 1.3. Preprocessing

### 1.3.1. "books" selecting data

In [7]:
books = books[['book_id', 'isbn', 'title', 'authors', 'average_rating', 'ratings_count']]

In [8]:
for i in range(len(books.authors)):
    books.authors[i] = int(list(books.authors[i][0].values())[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
books.replace('', np.nan, inplace=True)
books.dropna(inplace=True)

### 1.3.2. "ratings" selecting data

In [10]:
ratings = ratings[['user_id', 'book_id', 'rating']]

In [11]:
ratings.set_index('user_id', append=True)

,,book_id,rating
,user_id,,
0,8842281e1d1347389f2ab93d60773d4d,23310161,4
1,8842281e1d1347389f2ab93d60773d4d,17290220,5
2,8842281e1d1347389f2ab93d60773d4d,6954929,5
3,8842281e1d1347389f2ab93d60773d4d,460548,5
4,8842281e1d1347389f2ab93d60773d4d,11474551,5
...,...,...,...
734635,480ac8b4df39533a6c7fe6ae109aa56a,65112,2
734636,afcd096765a8c354be0706ab1324999b,555999,5
734637,d94c83867337514c94738b57a1d19677,389627,2


In [ ]:
# ratings.user_id = ratings.user_id.apply(lambda x: int(x, 16))

In [ ]:
# ratings = ratings[ratings.rating != 0]

### 1.3.3. cleaning

In [12]:
books = books[books.book_id.isin(ratings.book_id)].reset_index(drop=True)

In [13]:
ratings = ratings[ratings.book_id.isin(books.book_id)].reset_index(drop=True)

In [14]:
# books['book_id'] =  pd.factorize(books['book_id'])[0]

In [14]:
ratings['user_id'] = pd.factorize(ratings['user_id'])[0]

In [15]:
books

,book_id,isbn,title,authors,average_rating,ratings_count
0,287141,1599150603,The Aeneid for Boys and Girls,3041852,4.13,46
1,6066812,1934876569,All's Fairy in Love and War (Avalon: Web of Ma...,19158,4.22,98
2,89378,0590417010,Dog Heaven,5411,4.43,1331
3,3209312,0915190575,"Moths and Mothers, Feathers and Fathers: A Sto...",589328,4.29,11
4,1698376,1416904999,What Do You Do?,169159,3.57,23
...,...,...,...,...,...,...
103783,23346344,055256916X,Boy In The Tower,7336859,4.11,58
103784,823094,1575054035,"To Root, to Toot, to Parachute",44955,4.05,240
103785,7925060,0061960314,Instructions,1221698,4.29,40
103786,331839,0689852959,Jacqueline Kennedy Onassis: Friend of the Arts,10681,4.36,18


In [16]:
ratings

,user_id,book_id,rating
0,0,23310161,4
1,0,17290220,5
2,0,6954929,5
3,0,460548,5
4,0,11474551,5
...,...,...,...
680083,85972,65112,2
680084,85973,555999,5
680085,85974,389627,2
680086,85975,378,5


## 1.2. item dictionary 만들기

In [ ]:
#isbn_dict.to_csv(os.path.join(DIR, "isbn_dictionary.csv"), mode='w')

## 1.4. books, ratings 저장하기



In [ ]:
#ratings.to_csv(os.path.join(DIR, "emobook_ratings.csv"), mode='w')
#books.to_csv(os.path.join(DIR, "emobook_books.csv"), mode='w')

## 1.5. Data Set 만들기

### 1.5.1. training - test data 나누는 함수

In [17]:
def divide_sets(interactions, test_ratio=0.2, random_state=None):
    if random_state is None:
        random_state = np.random.RandomState()

    interactions = interactions.tocoo()

    #### data 섞기
    user_ids, item_ids, data = (interactions.row, interactions.col, interactions.data)

    shuffle_idx = np.arange(len(user_ids))
    random_state.shuffle(shuffle_idx)

    user_ids = user_ids[shuffle_idx]
    item_ids = item_ids[shuffle_idx]
    data = data[shuffle_idx]

    #### data 나누기
    slicing_point = int(test_ratio * len(user_ids))

    test_idx = slice(None, slicing_point)
    test = coo_matrix((data[test_idx], (user_ids[test_idx], item_ids[test_idx])), shape=interactions.shape, dtype=interactions.dtype)
    test = test.tocsr().tocoo()

    train_idx = slice(slicing_point, None)
    train = coo_matrix((data[train_idx], (user_ids[train_idx], item_ids[train_idx])), shape=interactions.shape, dtype=interactions.dtype)
    train = train.tocsr().tocoo()

    return test, train

### 1.5.2. data set 만들기

In [18]:
dataset = Dataset()

In [19]:
dataset.fit(users=ratings['user_id'], items=books['book_id'], item_features=books[books.columns[2:]].values.flatten())

In [20]:
item_features_list = [(books['book_id'][i],
                        [books['authors'][i],
                         books['average_rating'][i], books['ratings_count'][i]]) for i in range(books.shape[0])]
# 추가하면서 정확도 확인 !!

In [21]:
item_features = dataset.build_item_features(item_features_list)

In [ ]:
#with open(os.path.join(DIR, 'emobook_item_features.pickle'), 'wb') as f:
#    pickle.dump(item_features, f)

In [22]:
ratings_list = [(ratings['user_id'][i], ratings['book_id'][i]) for i in range(ratings.shape[0])]

In [23]:
interactions, weights = dataset.build_interactions(ratings_list)

In [24]:
test, train = divide_sets(interactions, test_ratio = 0.15)
train_weights = train.multiply(weights).tocoo()

In [ ]:
#np.save(os.path.join(DIR, 'test_user_ids'), test.row)

# 2. model 만들기

## 2.1. optimized parameter 찾기

In [ ]:
try:
    from skopt import forest_minimize
except:
    !pip install scikit-optimize
    from skopt import forest_minimize


def objective(params):
    # unpack
    epochs, learning_rate, no_components = params

    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components)
    model.fit(train, epochs=epochs,
              num_threads=4, verbose=True)
    
    patks = auc_score(model, test,num_threads=4)
    maptk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -maptk
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [ ]:
space = [(1, 100), # epochs
         (10**-4, 0.5, 'log-uniform'), # learning_rate
         (20, 200), # no_components
        ]

res_fm = forest_minimize(objective, space, n_calls=20,
                     random_state=0,
                     verbose=True)
print('Maximimum auc found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

---
## 2.2. 모델 만들기

In [25]:
# params = res_fm.x
sbjs = ['epochs', 'learning_rate', 'no_components']
params = [43, 0.030846147117754293, 114]

In [ ]:
with open(os.path.join(DIR, 'output/log_%s.txt'%(genre)),'w') as f:
    for (p, x) in zip(sbjs, params):
        print('{}: {}'.format(p, x), file=f)

In [26]:
start_time = time.time()
model = LightFM(no_components=params[2],  learning_schedule='adagrad', learning_rate=params[1], loss='warp', random_state=0)
model.fit(train, item_features=item_features, sample_weight=train_weights, epochs=params[0], num_threads=4, verbose=True)

Epoch: 100%|██████████| 43/43 [06:10<00:00,  8.63s/it]


In [ ]:
#with open(os.path.join(DIR, 'emobook_model.pickle'), 'wb') as f:
#f    pickle.dump(model, f)

# 3. 추천 목록 만들기

## 3.1. isbn dictionary 만들기

In [ ]:
#isbn_dict = pd.read_csv((os.path.join(DIR, "isbn_dictionary.csv")), index_col=0).dropna().reset_index(drop=True)
#isbn_dict = dict(zip(isbn_dict['book_id'],isbn_dict['isbn']))

## 3.2. 한 명 추천

In [27]:
scores = pd.Series(model.predict(32790,np.arange(books.book_id.count()), item_features=item_features), index=books.book_id).sort_values(ascending=False)

In [39]:
books[books['book_id']==20578964].isbn.values[0]

'0545646235'

In [40]:
def recommendation_user(user_id, threshold = 0,nrec_items = 20, show = True):
    
    n_items = books.book_id.count()
    scores = pd.Series(model.predict(user_id,np.arange(books.book_id.count()), item_features=item_features), index=books.book_id).sort_values(ascending=False)

    known_items = list(ratings[ratings['user_id'] == user_id].book_id.sort_values(ascending=False))

    predicted_items = []
    cnt=0
    for book_id in scores.index:
        if cnt >= nrec_items:
            break
        if book_id not in known_items:
            predicted_items.append((books[books['book_id']==book_id].isbn.values[0], scores[book_id]))
            cnt+=1
    
    # cnt = 0
    # predicted_items = []
    # for x in scores:
    #     if cnt == nrec_items:
    #         break
    #     if x not in known_items:
    #         try:
    #             predicted_items.append(int(isbn_dict[x]))
    #             cnt+=1
    #         except:
    #             pass

    return predicted_items

## 3.3. 여러 명 추천

In [41]:
def recommendation_users(user_ids, threshold = 0,nrec_items = 20, show = True):
    
    all_predicts = {}

    for user_id in user_ids:
        all_predicts[int(user_id)] = recommendation_user(int(user_id), threshold, nrec_items, show)

    return all_predicts

In [42]:
def recommendation(user_ids, threshold = 0,nrec_items = 20, show = True):
    if type(user_ids)==int:
        recomend_list = recommendation_user(user_ids)
    else:
        recomend_list = recommendation_users(user_ids)
    return recomend_list

In [43]:
my_user = ratings.groupby('user_id').size().sort_values(ascending=False).index[:100]

In [44]:
my_recommend_dict = recommendation(my_user)

In [45]:
my_recommend_dict

{56: [('1596434694', -5.793145),
  ('1452110069', -5.958697),
  ('1452141991', -5.960642),
  ('0062368435', -5.977023),
  ('0763669512', -6.002089),
  ('1452144249', -6.016619),
  ('1484712757', -6.080284),
  ('1454910631', -6.1270223),
  ('0375861815', -6.127552),
  ('0544340825', -6.139764),
  ('0544159039', -6.139951),
  ('1454904364', -6.1439657),
  ('1452101108', -6.1492805),
  ('0807530751', -6.1501055),
  ('0547641028', -6.165491),
  ('1442453362', -6.187773),
  ('0547773579', -6.1943502),
  ('0823425193', -6.2011275),
  ('1423145623', -6.2130303),
  ('0803741715', -6.2193117)],
 1828: [('1557091552', -4.584051),
  ('0439206472', -5.28969),
  ('0439206480', -5.418723),
  ('0811862712', -5.513216),
  ('0064408337', -5.670264),
  ('0689859376', -5.7620173),
  ('0689707495', -5.7740746),
  ('0007334060', -5.9012604),
  ('0723247714', -6.0524654),
  ('0448421658', -6.07068),
  ('0744583276', -6.168247),
  ('0448095017', -6.178703),
  ('1596436069', -6.2153597),
  ('1416909303', -6.2

In [46]:
with open(os.path.join(DIR, 'output/recommend_dict_%s.csv'%(genre)),'w') as f:
    print('user_id', 'isbn', 'score', sep=',', file=f)
    for user_id in my_recommend_dict.keys():
        for isbn, score in my_recommend_dict[user_id]:
            print(user_id, isbn, score, sep=',', file=f)

### 2.2.1. 정확도 측정

In [ ]:
item_features

In [ ]:
auc_train = auc_score(model, train, item_features = item_features).mean()
auc_test = auc_score(model, test, item_features = item_features).mean()

print("--- Run time:  {} mins ---".format((time.time() - start_time)/60))
print("Train AUC Score: {}".format(auc_train))
print("Test AUC Score: {}".format(auc_test))

In [ ]:
with open(os.path.join(DIR, 'output/log_%s.txt'%(genre)),'a') as f:
    print("Train AUC Score: {}".format(auc_train), file=f)
    print("Test AUC Score: {}".format(auc_test), file=f)

0
--- Run time:  7.042796397209168 mins ---
Train AUC Score: 0.9776267409324646
Test AUC Score: 0.8526195287704468

In [ ]:
# item_biases, item_embeddings = model.get_item_representations(features=item_features)